In [1]:
#widgets
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2
import motors

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

In [2]:
# TODO: Check for and set the actual parameters for the camera device.
cam_hfov = 101
cam_vfov = 68

In [3]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11l_half.engine")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [4]:
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
from traitlets.config.configurable import SingletonConfigurable

# Define a Camera class that inherits from SingletonConfigurable
class Camera(SingletonConfigurable):
    color_value = traitlets.Any() # monitor the color_value variable
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera has opened succesfully
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

         # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        #flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self): #For data capturing only

        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                self.color_value= cv2.cvtColor(self.color_value, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())

    def pixel_to_angle(self, x, y, hfov=cam_hfov, vfov=cam_vfov):
        """
        Calculate horizontal, vertical, and Euclidean viewing angles for a given pixel.

        Parameters:
            x (float): Pixel x-coordinate
            y (float): Pixel y-coordinate
            width (int): Image width in pixels
            height (int): Image height in pixels
            hfov (float): Horizontal field of view in degrees
            vfov (float): Vertical field of view in degrees
    
        Returns:
            (h_angle, v_angle, euclidean_angle): Tuple of three angles in degrees
        """
        cx, cy = self.width / 2, self.height / 2
    
        h_angle = ((x - cx) / cx) * (hfov / 2)
        v_angle = ((y - cy) / cy) * (vfov / 2)
    
        euclidean_angle = math.sqrt(h_angle**2 + v_angle**2)
    
        if x < cx:
            euclidean_angle = -euclidean_angle
    
        return h_angle #, v_angle, euclidean_angle
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera()
camera.start() # start capturing the data


[2025-11-14 11:47:59 UTC][ZED][INFO] Logging level INFO
[2025-11-14 11:47:59 UTC][ZED][INFO] Logging level INFO
[2025-11-14 11:47:59 UTC][ZED][INFO] Logging level INFO
[2025-11-14 11:48:00 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-11-14 11:48:01 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-11-14 11:48:01 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-11-14 11:48:01 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-11-14 11:48:01 UTC][ZED][INFO] [Init]  Serial Number: S/N 33971626
Loading yolo11l_half.engine for TensorRT inference...


Exception in thread Thread-5 (_capture_frames):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_5158/1568382331.py", line 57, in _capture_frames
  File "/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py", line 716, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py", line 706, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py", line 1513, in _notify_trait
    self.notify_change(
  File "/usr/local/lib/python3.10/dist-packages/traitlets/traitlets.py", line 1525, in notify_change
    return self._notify_observe

Type:  <class 'float'> Value 35.470238095238095
Turning_speed =  -0.1223193794670531
Turning:  -0.3057984486676327
turning right


In [5]:
def detect_human(frame):
    """Common function to detect humans and return closest human info"""
    results = model(frame, verbose=False)
    
    conf_threshold = 0.5
    human_detected = False
    min_distance = float('inf')
    closest_bbox = None
    
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:  # Human subject
                if result.boxes.conf[i] > conf_threshold:
                    human_detected = True
                    bbox = result.boxes.xyxy[i]
                    
                    # Calculate center of bounding box
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get depth at human center
                    if (0 <= center_y < camera.height and 0 <= center_x < camera.width):
                        distance = camera.depth_image[center_y, center_x]
                        
                        if not np.isnan(distance) and distance > 0:
                            if distance < min_distance:
                                min_distance = distance
                                closest_bbox = bbox
                                
    
    return human_detected, min_distance, closest_bbox, [center_x, center_y]

In [6]:
class SpringFollower:
    def __init__(self, speed=0.5, k=10.0, b=6.0, rest_length=0.5, dt=0.02):
        self.speed = speed
        self.motors = motors.MotorsYukon(mecanum=False)

        self.was_turning_right = True
                                         
        self.k = k
        self.b = b
        self.l0 = rest_length
        self.dt = dt

        self.vel = np.array([0.0, 0.0])
        self._origin = np.array([0.0, 0.0])

    def human_position(self, r, theta):
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        return np.array([x, y])

    def update(self, human_position):

        distance = human_position[0]
        angle = human_position[1]
        
        ph = self.human_position(human_position[0], human_position[1])
        diff = ph - self._origin
        dist = np.linalg.norm(diff)

        if dist > 1e-6:
            vel_spring = (self.k / self.b) * (1 - self.l0 / dist) * diff
        else:
            vel_spring = np.zeros(2)

        self.vel = vel_spring / self.b
        return self.vel

    def drive(self, human_detected, angle):
        # max angle is 90 degrees
        # at -90 and 90 degrees wheel multiplier should be 0 for corresponding side
        # as robot is turning theta decreeses
        print("Type: ", type(angle), "Value", angle)
        angle_clipped = angle
        np.clip(angle_clipped, 1, 1)

        if not human_detected:
            if self.was_turning_right:
                angle_clipped = math.radians(90)
            else:
                angle_clipped = math.radians(-90)
                
        turning = 0.5*np.cos(angle)

        turning_speed = turning*self.speed
        print("Turning_speed = ", turning_speed)
        
        print("Turning: ", turning)
        if angle_clipped > 0:
            print("turning right")
            self.motors.frontLeft(speed=self.speed)
            self.motors.backLeft(speed=self.speed)

            self.motors.frontRight(speed=turning_speed)
            self.motors.backRight(speed=turning_speed)

            return True

        elif angle_clipped < 0:            
            print("turning left")

            self.motors.frontLeft(speed=turning_speed)
            self.motors.backLeft(speed=turning_speed)

            self.motors.frontRight(speed=self.speed)
            self.motors.backRight(speed=self.speed)

        else:
            self.motors.frontLeft(speed=self.speed)
            self.motors.backLeft(speed=self.speed)

            self.motors.frontRight(speed=self.speed)
            self.motors.backRight(speed=self.speed)

            return False
    
    def move(self, angle):
        self.drive(angle)    
    
    
    def stop(self):
        # turn of cameras and such
        self.motors.stop()
            


In [7]:

#create follower
follower = SpringFollower(speed=0.4)

In [8]:
# Create text input widget for keyboard control
text_input = widgets.Text(
    value='',
    placeholder='Type: l=Love, f=Fear, a=Aggressive, c=Curious, s=Stop',
    description='Behavior:',
    disabled=False
)

def on_text_change(change):
    """Handle keyboard input to switch behaviors"""
    global current_behavior
    input_value = change['new']
    
    if len(input_value) > 0:
        last_char = input_value[-1].lower()
        
        if last_char == 'l':
            current_behavior = 'love'
            print('\nSwitched to LOVE behavior')
        elif last_char == 'f':
            current_behavior = 'fear'
            print('\nSwitched to FEAR behavior')
        elif last_char == 'a':
            current_behavior = 'aggressive'
            print('\nSwitched to AGGRESSIVE behavior')
        elif last_char == 'c':
            current_behavior = 'curious'
            print('\nSwitched to CURIOUS behavior')
        elif last_char == 's':
            current_behavior = 'stop'
            robot.stop()
            print('\nSTOPPED - Robot halted')

text_input.observe(on_text_change, names='value')
display(text_input)

def main_callback(change):
    """Main callback that executes current behavior"""    
    frame = change['new']
    
    # Prepare depth colormap for display
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(camera.depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    # Detect human
    human_detected, distance, bbox, center = detect_human(frame)
    
    angle = camera.pixel_to_angle(center[0], center[1])

    follower.drive(human_detected, angle)    
    
    # Display images
    scale = 0.1
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

# Attach callback to camera
camera.observe(main_callback, names=['color_value'])

print('\n=== Braitenberg Behaviors Active ===')
print('Press keys to switch behaviors:')
print('  l = Love (follow human)')
print('  f = Fear (retreat from human)')
print('  a = Aggressive (charge at human)')
print('  c = Curious (investigate human)')
print('  s = Stop (halt robot)')
print('===================================')




Text(value='', description='Behavior:', placeholder='Type: l=Love, f=Fear, a=Aggressive, c=Curious, s=Stop')


=== Braitenberg Behaviors Active ===
Press keys to switch behaviors:
  l = Love (follow human)
  f = Fear (retreat from human)
  a = Aggressive (charge at human)
  c = Curious (investigate human)
  s = Stop (halt robot)


In [9]:
#follower.stop()
#camera.stop()